In [1]:
# 블랜더챗봇 & GPT2파인튜닝

In [2]:
#환경구축
!pip install torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio===0.9.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.3 MB/s eta 0:16:02tcmalloc: large alloc 1147494400 bytes == 0x3a42e000 @  0x7fed8ddc7615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████▌               | 1055.7 MB 1.3 MB/s eta 0:12:11tcmalloc: large alloc 1434370048 bytes == 0x7ea84000 @  0x7fed8ddc7615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████

In [3]:
#트랜스포머 설치
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 12.6 MB/s 
     |████████████████████████████████| 120 kB 70.0 MB/s 
     |████████████████████████████████| 6.6 MB 69.8 MB/s 


In [4]:
#블랜더 임포트
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration

In [5]:
#블랜더 토크나이저, 모델 설정
tokenizer = BlenderbotTokenizer.from_pretrained("facebook/blenderbot-400M-distill") #디스틸-lite모델

model = BlenderbotForConditionalGeneration.from_pretrained("facebook/blenderbot-400M-distill")

In [7]:
sentence = "I want to go the Mars"

In [8]:
#토크나이징
inputs = tokenizer(sentence, return_tensors="pt")

In [9]:
#모델에 input해서 결과 받음
results = model.generate(**inputs)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 60 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [10]:
#결과
results

tensor([[   1,  281,  422,  913,  287,  425,  287, 7863,  371,  731,   21,  452,
          315,  271, 1205,   20,   83,  305, 2415, 4280,  302,  271, 5680,  305,
         4243, 1228,   21,    2]])

In [11]:
#결과 디코딩
tokenizer.decode(results[0])

'<s> I would love to go to Mars as well. It is the second-largest planet in the Solar System.</s>'

In [ ]:
# GPT2 파인튜닝

In [12]:
#gpt2 토크나이저 임포트
from transformers import GPT2Tokenizer

In [13]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [14]:
train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [15]:
#데이터 불러오기
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [16]:
#모델 정의 & 가져오기
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=8, # batch size for training
    per_device_eval_batch_size=12,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1002: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [17]:
#학습
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14085
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5283


Step,Training Loss
500,2.933100
1000,2.662000
1500,2.573400
2000,2.472000
2500,2.393800
3000,2.368100
3500,2.355000
4000,2.267800
4500,2.256100
5000,2.254000


Saving model checkpoint to ./gpt2-finetuned/checkpoint-800
Configuration saved in ./gpt2-finetuned/checkpoint-800/config.json
Model weights saved in ./gpt2-finetuned/checkpoint-800/pytorch_model.bin
Saving model checkpoint to ./gpt2-finetuned/checkpoint-1600
Configuration saved in ./gpt2-finetuned/checkpoint-1600/config.json
Model weights saved in ./gpt2-finetuned/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to ./gpt2-finetuned/checkpoint-2400
Configuration saved in ./gpt2-finetuned/checkpoint-2400/config.json
Model weights saved in ./gpt2-finetuned/checkpoint-2400/pytorch_model.bin
Saving model checkpoint to ./gpt2-finetuned/checkpoint-3200
Configuration saved in ./gpt2-finetuned/checkpoint-3200/config.json
Model weights saved in ./gpt2-finetuned/checkpoint-3200/pytorch_model.bin
Saving model checkpoint to ./gpt2-finetuned/checkpoint-4000
Configuration saved in ./gpt2-finetuned/checkpoint-4000/config.json
Model weights saved in ./gpt2-finetuned/checkpoint-4000/pytorch_mod

TrainOutput(global_step=5283, training_loss=2.4426029226262305, metrics={'train_runtime': 632.2899, 'train_samples_per_second': 66.829, 'train_steps_per_second': 8.355, 'total_flos': 2760223703040000.0, 'train_loss': 2.4426029226262305, 'epoch': 3.0})

In [18]:
#모델 저장
trainer.save_model()

Saving model checkpoint to ./gpt2-finetuned
Configuration saved in ./gpt2-finetuned/config.json
Model weights saved in ./gpt2-finetuned/pytorch_model.bin


In [19]:
#인퍼런스

from transformers import pipeline

generator = pipeline('text-generation', model='./gpt2-finetuned', tokenizer='gpt2')

loading configuration file ./gpt2-finetuned/config.json
Model config GPT2Config {
  "_name_or_path": "./gpt2-finetuned",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.21.2",
  "use_cache": true,
  "vocab_size": 50257
}



storing https://huggingface.co/gpt2/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/16a2f78023c8dc511294f0c97b5e10fde3ef9889ad6d11ffaa2a00714e73926e.cf2d0ecb83b6df91b3dbb53f1d1e4c311578bfd3aa0e04934215a49bf9898df0
creating metadata file for /root/.cache/huggingface/transformers/16a2f78023c8dc511294f0c97b5e10fde3ef9889ad6d11ffaa2a00714e73926e.cf2d0ecb83b6df91b3dbb53f1d1e4c311578bfd3aa0e04934215a49bf9898df0
loading file https://huggingface.co/gpt2/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/684fe667923972fb57f6b4dcb61a3c92763ad89882f3da5da9866baf14f2d60f.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f
loading file https://huggingface.co/gpt2/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/c0c761a63004025aeadd530c4c27b860ec4ecbe8a00531233de21d865a402598.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/gpt2/resolve/main/tokenizer.jso

In [20]:
generator('Zuerst Hähnchen')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 50 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


[{'generated_text': 'Zuerst Hähnchen! Well-tried, if a\n     genius could work that method on such a moorland creature. I heard some\n     very vague things, and it struck me that'}]